## A Tiny Interpreter
- LOAD_VALUE
- ADD_TWO__VALUES
- PRINT_ANSWER

In [1]:
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),  # the first number
                     ("LOAD_VALUE", 1),  # the second number
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)],
    "numbers": [7, 5] }

In [2]:
class Interpreter:
    def __init__(self):
        self.stack = []
    
    def LOAD_VALUE(self, number):
        self.stack.append(number)
    
    def PRINT_ANSWER(self):
        answer = self.stack.pop()
        print(answer)
    
    def ADD_TWO_VALUES(self):
        first_num = self.stack.pop()
        second_num = self.stack.pop()
        total = first_num + second_num
        self.stack.append(total)
        
    def run_code(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        numbers = what_to_execute["numbers"]
        for each_step in instructions:
            instruction, argument = each_step
            if instruction == "LOAD_VALUE":
                number = numbers[argument]
                self.LOAD_VALUE(number)
            elif instruction == "ADD_TWO_VALUES":
                self.ADD_TWO_VALUES()
            elif instruction == "PRINT_ANSWER":
                self.PRINT_ANSWER() 

In [3]:
interpreter = Interpreter()
interpreter.run_code(what_to_execute)

12


In [4]:
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),
                     ("LOAD_VALUE", 1),
                     ("ADD_TWO_VALUES", None),
                     ("LOAD_VALUE", 2),
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)],
    "numbers": [7, 5, 8] }

In [5]:
interpreter = Interpreter()
interpreter.run_code(what_to_execute)

20


### Variables
Variables require an instruction for storing the value of a variable, STORE_NAME

In [6]:
def s():
...     a = 1
...     b = 2
...     print(a + b)
# a friendly compiler transforms `s` into:
    what_to_execute = {
        "instructions": [("LOAD_VALUE", 0),
                         ("STORE_NAME", 0),
                         ("LOAD_VALUE", 1),
                         ("STORE_NAME", 1),
                         ("LOAD_NAME", 0),
                         ("LOAD_NAME", 1),
                         ("ADD_TWO_VALUES", None),
                         ("PRINT_ANSWER", None)],
        "numbers": [1, 2],
        "names":   ["a", "b"] }

In [7]:
class Interpreter:
    def __init__(self):
        self.stack = []
        self.environment = {}

    def STORE_NAME(self, name):
        val = self.stack.pop()
        self.environment[name] = val

    def LOAD_NAME(self, name):
        val = self.environment[name]
        self.stack.append(val)

    def parse_argument(self, instruction, argument, what_to_execute):
        """ Understand what the argument to each instruction means."""
        numbers = ["LOAD_VALUE"]
        names = ["LOAD_NAME", "STORE_NAME"]

        if instruction in numbers:
            argument = what_to_execute["numbers"][argument]
        elif instruction in names:
            argument = what_to_execute["names"][argument]

        return argument

    def run_code(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)

            if instruction == "LOAD_VALUE":
                self.LOAD_VALUE(argument)
            elif instruction == "ADD_TWO_VALUES":
                self.ADD_TWO_VALUES()
            elif instruction == "PRINT_ANSWER":
                self.PRINT_ANSWER()
            elif instruction == "STORE_NAME":
                self.STORE_NAME(argument)
            elif instruction == "LOAD_NAME":
                self.LOAD_NAME(argument)
                
    def execute(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)
            bytecode_method = getattr(self, instruction)
            if argument is None:
                bytecode_method()
            else:
                bytecode_method(argument)

## Real Python Bytecode

In [8]:
def cond():
...     x = 3
...     if x < 5:
...         return 'yes'
...     else:
...         return 'no'

In [9]:
cond.__code__.co_code

b'd\x01}\x00|\x00d\x02k\x00r\x10d\x03S\x00d\x04S\x00d\x00S\x00'

In [10]:
list(cond.__code__.co_code)

[100,
 1,
 125,
 0,
 124,
 0,
 100,
 2,
 107,
 0,
 114,
 16,
 100,
 3,
 83,
 0,
 100,
 4,
 83,
 0,
 100,
 0,
 83,
 0]

In [11]:
import dis
dis.dis(cond)

  2           0 LOAD_CONST               1 (3)
              2 STORE_FAST               0 (x)

  3           4 LOAD_FAST                0 (x)
              6 LOAD_CONST               2 (5)
              8 COMPARE_OP               0 (<)
             10 POP_JUMP_IF_FALSE       16

  4          12 LOAD_CONST               3 ('yes')
             14 RETURN_VALUE

  6     >>   16 LOAD_CONST               4 ('no')
             18 RETURN_VALUE
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


- The first column show the line nuber
- The second column is an index into the bytecode
- The third column is the instructin itself, mapped to its human-readable name.
- The fourth column, when present, is the argument to that instruction.
- The fifth column, when present, is a hint about what the argument means.

In [12]:
# mapping from bytes to intelligible strings
dis.opname[100]

'LOAD_CONST'

In [13]:
dis.opname[125]

'STORE_FAST'

## Conditionals and Loops
Python handles loops and conditionals with GOTO statements in the bytecode!

In [14]:
dis.dis(cond)

  2           0 LOAD_CONST               1 (3)
              2 STORE_FAST               0 (x)

  3           4 LOAD_FAST                0 (x)
              6 LOAD_CONST               2 (5)
              8 COMPARE_OP               0 (<)
             10 POP_JUMP_IF_FALSE       16

  4          12 LOAD_CONST               3 ('yes')
             14 RETURN_VALUE

  6     >>   16 LOAD_CONST               4 ('no')
             18 RETURN_VALUE
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


The instruction POP_JUMP_IF_FALSE is responsible for implementing the if. This instruction will pop the top value off the interpreter's stack. 
- if the value is true, then nothing happends.
- if the value is false, then the interpreter will jump to another instruction.

Loops also reply on jumping.

In [15]:
def loop():
...      x = 1
...      while x < 5:
...          x = x + 1
...      return x

In [16]:
dis.dis(loop)

  2           0 LOAD_CONST               1 (1)
              2 STORE_FAST               0 (x)

  3           4 SETUP_LOOP              20 (to 26)
        >>    6 LOAD_FAST                0 (x)
              8 LOAD_CONST               2 (5)
             10 COMPARE_OP               0 (<)
             12 POP_JUMP_IF_FALSE       24

  4          14 LOAD_FAST                0 (x)
             16 LOAD_CONST               1 (1)
             18 BINARY_ADD
             20 STORE_FAST               0 (x)
             22 JUMP_ABSOLUTE            6
        >>   24 POP_BLOCK

  5     >>   26 LOAD_FAST                0 (x)
             28 RETURN_VALUE


## Frame
A frame is a collection of information and context for a chunk of code.

In [17]:
def bar(y):
...     z = y + 3     # <--- (3) ... and the interpreter is here.
...     return z

In [18]:
def foo():
...     a = 1
...     b = 2
...     return a + bar(b)

In [19]:
foo()

6

![The Call Stack](./PythonInterpreter/callStack.jpg)

## Byterun

## Reference

- [A Python Interpreter Written in Python
](http://aosabook.org/en/500L/a-python-interpreter-written-in-python.html)